<a href="https://colab.research.google.com/github/RedBatProject/machine-vision-w/blob/main/work_87_leaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 95.71% of tomatoes

In [3]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"willyvonka","key":"60045de61af3e61a515ab858811b8e46"}'}

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
#!/bin/bash
!kaggle datasets download emmarex/plantdisease

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 99% 653M/658M [00:04<00:00, 124MB/s]
100% 658M/658M [00:04<00:00, 146MB/s]


In [ ]:
!unzip /content/plantdisease.zip

In [8]:
import os

In [14]:
import shutil

In [16]:
for i,j,k in os.walk('/content/plantvillage/PlantVillage/'):
    print(i)
    if 'Pepper' in i:
        shutil.rmtree(i)

/content/plantvillage/PlantVillage/
/content/plantvillage/PlantVillage/Tomato_healthy
/content/plantvillage/PlantVillage/Tomato_Septoria_leaf_spot
/content/plantvillage/PlantVillage/Tomato_Bacterial_spot
/content/plantvillage/PlantVillage/Tomato_Late_blight
/content/plantvillage/PlantVillage/Tomato_Early_blight
/content/plantvillage/PlantVillage/Tomato_Leaf_Mold
/content/plantvillage/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus
/content/plantvillage/PlantVillage/Pepper__bell___healthy
/content/plantvillage/PlantVillage/Tomato__Tomato_mosaic_virus
/content/plantvillage/PlantVillage/Tomato_Spider_mites_Two_spotted_spider_mite
/content/plantvillage/PlantVillage/Pepper__bell___Bacterial_spot
/content/plantvillage/PlantVillage/Tomato__Target_Spot


In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load the dataset from the directory
dataset_path = "/content/plantvillage/PlantVillage"
batch_size = 32
img_height = 224  # Adjust the height as needed
img_width = 224   # Adjust the width as needed

# Load training dataset
train_dataset = image_dataset_from_directory(
    dataset_path,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',  # 'int' for integer labels or 'categorical' for one-hot encoding
    shuffle=True,      # Shuffle the dataset
    seed=123,          # Set a seed for reproducibility
    validation_split=0.2,  # Use 20% of the data for validation
    subset='training'  # 'training' subset
)

# Load validation dataset
validation_dataset = image_dataset_from_directory(
    dataset_path,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',  # Same as above
    shuffle=True,
    seed=123,
    validation_split=0.2,  # Same as above
    subset='validation'  # 'validation' subset
)

Found 16011 files belonging to 10 classes.
Using 12809 files for training.
Found 16011 files belonging to 10 classes.
Using 3202 files for validation.


In [20]:
class_names = len(train_dataset.class_names)

In [21]:
hw = 224
wh = 224

In [26]:

# Define a simple dense model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(hw, wh, 3)),  # Flatten the 64x64x3 images
    tf.keras.layers.Dense(128, activation='relu'),     # Fully connected layer
    tf.keras.layers.Dense(64, activation='relu'),      # Another fully connected layer
    tf.keras.layers.Dense(class_names, activation='softmax')     # Output layer with 7 classes (softmax for classification)
])

# Compile the model
model.compile(optimizer='adam',
              loss='spars_categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' as labels are integers
              metrics=['accuracy'])

# Print a summary of the model
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)                  │ (None, 150528)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │      19,267,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,276,618 (73.53 MB)

 Trainable params: 19,276,618 (73.53 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=10,verbose=1)

Epoch 1/10


ValueError: Could not interpret loss identifier: spars_categorical_crossentropy